In [200]:
import helpers
import numpy as np
import re
import math
from tqdm import tqdm
import copy
import networkx as nx
from collections import Counter
import ast
import itertools as it
from shapely.geometry import Polygon

data = """#############
#...........#
###D#D#C#C###
  #B#A#B#A#
  #########"""

#############
#...........#
###B#C#B#D###
  #A#D#C#A#
  #########

#with open("input_23.txt", 'r') as f:
#    data = f.read()

data = data.lstrip()
data = data.rstrip()
    
    
#changeDict = {"forward": np.array([1,0]), "down": np.array([0,1]), "up": np.array([0,-1])}
#data = re.findall("(\w+) x=([-+]?\d+)..([-+]?\d+),y=([-+]?\d+)..([-+]?\d+),z=([-+]?\d+)..([-+]?\d+)", data)
#data = [x.rstrip() for x in data.split("\n\n") if x]
data


'#############\n#...........#\n###D#D#C#C###\n  #B#A#B#A#\n  #########'

In [201]:
move = {"A": 1, "B": 10, "C": 100, "D": 1000}
move_num = {1: 1, 2: 10, 3: 100, 4: 1000}
translation = {1: "A", 2: "B", 3: "C", 4: "D"}
exits = {0: 2, 1: 4, 2: 6, 3: 8}

In [202]:
hallway = np.zeros(11, dtype="int8")
hallway[[2,4,6,8]] = -1
#hallway[[5]] = 1
#hallway[[10]] = 3
print(hallway)
Aroom = [4, 2]
Broom = [4, 1]
Croom = [3, 2]
Droom = [3, 1]
#Aroom = [2, 1]
#Broom = [3, 4]
#Croom = [2, 3]
#Droom = [4, 1]

rooms = [Aroom, Broom, Croom, Droom]

[ 0  0 -1  0 -1  0 -1  0 -1  0  0]


In [203]:
scores = []
def recurse(hallway, rooms, score):
    dests = []
    if rooms[0] == [1,1] and rooms[1] == [2,2] and rooms[2] == [3,3] and rooms[3] == [4,4]:
        #print(f"score: {score}")
        scores.append(score)
    #print("---rooms---")
    for i, x in enumerate(rooms):
        if (not x[0] == (i+1) or not x[1] == (i+1)) and not x[0] == 0:
            d = destinations((x[0], i), hallway, rooms, 1)
            if d:
                dests.append([(i,0), d])
        elif x[0] == 0 and not x[1] == (i+1) and not x[1] == 0:
            d2 = destinations((x[1], i), hallway, rooms, 2)
            if d2:
                dests.append([(i,1), d2])
    #print("---Halways---")
    for i, x in enumerate(hallway):
        if x > 0:
            d3 = destinations((x, i), hallway, rooms, 0)
            if d3:
                dests.append([(-1,i), d3])
    #return dests
    for s_pos, dests_l in dests:
        for l in dests_l:
            e_pos, steps = l
            nhallway = hallway.copy()
            nrooms = copy.deepcopy(rooms)
                
            if s_pos[0] == -1:
                val = nhallway[s_pos[1]]
                nhallway[s_pos[1]] = 0
            elif s_pos[0] > -1:
                val = nrooms[s_pos[0]][s_pos[1]]
                nrooms[s_pos[0]][s_pos[1]] = 0
                
            if e_pos[0] == -1:
                nhallway[e_pos[1]] = val
            elif e_pos[0] > -1:
                nrooms[e_pos[0]][e_pos[1]] = val
            recurse(nhallway, nrooms, score + (steps * move_num[val]))
                         
            
# -1: hallway
# 0 : Aroom
# 1: Broom
# 2: Croom
# 3: Droom
def destinations(pos, hallway, rooms, steps):
    #print(pos)
    if steps > 0:
        startLocation = exits[pos[1]]
    else:
        startLocation = pos[1]
    destinations = []
    accessible = set()
    for x in range(startLocation+1, 11): #+1
        if not hallway[x] > 0:
            accessible.add(x)
        else:
            break
    for x in range(startLocation-1, -1, -1): #-1
        if not hallway[x] > 0:
            accessible.add(x)
        else:
            break
    if exits[pos[0]-1] in accessible:
        if rooms[pos[0]-1][1] == pos[0] and rooms[pos[0]-1][0] == 0:
            #steps += abs(startLocation - exits[pos[0]-1]) + 1
            destinations.append([(pos[0]-1,0), steps + abs(startLocation - exits[pos[0]-1]) + 1])
            return destinations
        elif rooms[pos[0]-1][1] == 0 and rooms[pos[0]-1][0] == 0:
            #steps += abs(startLocation - exits[pos[0]-1]) + 2
            destinations.append([(pos[0]-1,1), steps + abs(startLocation - exits[pos[0]-1]) + 2])
            return destinations
    if steps > 0:
        accessible.difference_update({2,4,6,8})
        for x in accessible:
            destinations.append([(-1,x), steps + abs(startLocation - x)])
    return destinations

    

In [204]:
recurse(hallway, rooms, 0)

KeyboardInterrupt: 

In [205]:
min(scores)

16059